In [ ]:
from pathlib import Path

In [ ]:
import pandas as pd


In [ ]:
import numpy as np


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, average_precision_score, classification_report, auc

In [ ]:
np.random.seed(42)

In [ ]:
import time

In [ ]:
import zipfile

In [ ]:
zip_path = "/content/combined_danmini.zip"

In [ ]:
extract_path = "/content/Danmini"

In [ ]:
if not Path(extract_path).exists():
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print("Dataset unzipped.")
else:
    print("Dataset already unzipped.")

Dataset unzipped.


In [ ]:
dataFrame = pd.read_csv("/content/Danmini/combined_danmini.csv")


In [ ]:
#The Random Forest implementation

In [ ]:
from sklearn.ensemble import RandomForestClassifier


In [ ]:
X = dataFrame.drop(columns="label", axis=1)
y = dataFrame["label"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

In [ ]:
rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=None,
    random_state=42,
    n_jobs=-1
)

In [ ]:
start_train = time.time()
rf.fit(X_train, y_train)
end_train = time.time()


print(f"\nTraining time: {end_train - start_train:.3f} seconds")


Training time: 46.253 seconds


In [ ]:
#This part will be for the robustness, I will do 3 types of noise/robustness transformations

In [ ]:
#The first is noise (Gaussian Noise)

In [ ]:
noise = 0.2

In [ ]:
X_test_gaussian = X_test + np.random.normal(0, noise, X_test.shape)


In [ ]:
y_pred_gauss = rf.predict(X_test_gaussian)
y_prob_gauss = rf.predict_proba(X_test_gaussian)



In [ ]:
gauss_f1 = f1_score(y_test, y_pred_gauss, average="weighted")

In [ ]:
gauss_pr_auc = average_precision_score(y_test, y_prob_gauss, average='macro')



In [ ]:
gauss_roc_auc = roc_auc_score(y_test, y_prob_gauss, multi_class='ovr')


In [ ]:
start = time.time()
y_pred = rf.predict(X_test_gaussian)
inference_time_total = time.time() - start
inference_latency_per_sample = inference_time_total / X_test_gaussian.shape[0]

In [ ]:
print("Gaussian Noise Test Results for Random Forest ")
print("Noise " , noise)
print("F1 Score:", gauss_f1)
print("PR AUC:", gauss_pr_auc)
print("ROC AUC:", gauss_roc_auc)
print("Interference Latency: ", inference_latency_per_sample)

Gaussian Noise Test Results for Random Forest 
Noise  0.2
F1 Score: 0.08233788925699959
PR AUC: 0.9955372653058978
ROC AUC: 0.9981210601664259
Interference Latency:  1.1067533493041992e-05


In [ ]:
#The second is mean shift drift

In [ ]:
drift_strength = 0.2

In [ ]:
X_test_drift = X_test + drift_strength


In [ ]:
y_pred_drift = rf.predict(X_test_drift)
y_prob_drift = rf.predict_proba(X_test_drift)

In [ ]:
drift_f1 = f1_score(y_test, y_pred_drift, average="weighted")


In [ ]:
drift_pr_auc = average_precision_score(y_test, y_prob_drift, average='macro')


In [ ]:
drift_roc_auc = roc_auc_score(y_test, y_prob_drift, multi_class='ovr')


In [ ]:
start = time.time()
y_pred = rf.predict(X_test_drift)
inference_time_total = time.time() - start
inference_latency_per_sample = inference_time_total / X_test_drift.shape[0]

In [ ]:
print("Drift Noise Test Results for Random Forest ")
print("Drift strength " , drift_strength)
print("F1 Score:", drift_f1)
print("PR AUC:", drift_pr_auc)
print("ROC AUC:", drift_roc_auc)
print("Interference Latency: ", inference_latency_per_sample)

Drift Noise Test Results for Random Forest 
Drift strength  0.2
F1 Score: 0.07933626408071391
PR AUC: 0.9995354525808536
ROC AUC: 0.9996687203265698
Interference Latency:  5.896290143330892e-06


In [ ]:
#The third is adversarial perturbation (adversarial FGSM)

In [ ]:
epsilon = 0.2

In [ ]:
rf_probs = rf.predict_proba(X_test)[:, 1]


In [ ]:
direction = np.where(rf_probs > 0.5, 1, -1).reshape(-1, 1)

In [ ]:
perturbation = epsilon * direction


In [ ]:
X_test_adv = X_test + perturbation


In [ ]:
y_pred_adv = rf.predict(X_test_adv)


In [ ]:
adv_f1 = f1_score(y_test, y_pred_adv, average='weighted')


In [ ]:
y_prob_adv = rf.predict_proba(X_test_adv)


In [ ]:
adv_roc_auc = roc_auc_score(y_test, y_prob_adv, multi_class="ovr")


In [ ]:
adv_pr_auc = average_precision_score(y_test, y_prob_adv, average="macro")


In [ ]:
start = time.time()
y_pred = rf.predict(X_test_adv)
inference_time_total = time.time() - start
inference_latency_per_sample = inference_time_total / X_test_adv.shape[0]

In [ ]:
print("Adversarial Noise Test Results for Random Forest ")
print("Epsilon " , epsilon)
print("F1 Score:", adv_f1)
print("PR AUC:", adv_pr_auc)
print("ROC AUC:", adv_roc_auc)
print("Interference Latency: ", inference_latency_per_sample)

Adversarial Noise Test Results for Random Forest 
Epsilon  0.2
F1 Score: 0.08658910244528385
PR AUC: 0.9999865722455806
ROC AUC: 0.9999471290174666
Interference Latency:  6.063302357991537e-06


In [ ]:
#Naive Bayes Gaussian implementation

In [ ]:
dataFrame2 = pd.read_csv("/content/Danmini/combined_danmini.csv")

In [ ]:
X = dataFrame2.drop(columns="label", axis=1)
y = dataFrame2["label"]

In [ ]:
from sklearn.naive_bayes import GaussianNB


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

In [ ]:
model_nb = GaussianNB()

In [ ]:
model_nb.fit(X_train, y_train)

GaussianNB()

In [ ]:
#Gaussian noise for Gaussian naive bayes

In [ ]:
noise_level = 0.2
noise = np.random.normal(0, noise_level, X_test.shape)
X_test_gauss = X_test + noise

In [ ]:
start = time.time()
y_pred_gauss = model_nb.predict(X_test_gauss)
inference_time_total = time.time() - start
inference_latency = inference_time_total / X_test_gauss.shape[0]

In [ ]:
y_prob_gauss = model_nb.predict_proba(X_test_gauss)


In [ ]:
gauss_f1 = f1_score(y_test, y_pred_gauss, average="weighted")

In [ ]:
gauss_pr_auc = average_precision_score(y_test, y_prob_gauss, average='macro')


In [ ]:
gauss_roc_auc = roc_auc_score(y_test, y_prob_gauss, multi_class='ovr')


In [ ]:
print("\nGaussian Noise Test Results for GaussianNB")
print("Noise level:", noise_level)
print("F1:", gauss_f1)
print("PR-AUC:", gauss_pr_auc)
print("ROC-AUC:", gauss_roc_auc)
print("Inference latency:", inference_latency)



Gaussian Noise Test Results for GaussianNB
Noise level: 0.2
F1: 0.012308327663151718
PR-AUC: 0.4437057324402103
ROC-AUC: 0.7004222996100484
Inference latency: 1.9759019215901694e-06


In [ ]:
#Drift for Gaussian NB

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

In [ ]:
drift_strength = 0.2
X_test_drift = X_test + drift_strength

In [ ]:
y_prob_drift = model_nb.predict_proba(X_test_drift)


In [ ]:
start = time.time()
y_pred_drift = model_nb.predict(X_test_drift)
inference_time_total = time.time() - start
inference_latency = inference_time_total / X_test_drift.shape[0]


In [ ]:
drift_f1 = f1_score(y_test, y_pred_drift, average="weighted")


In [ ]:
drift_pr_auc = average_precision_score(y_test, y_prob_drift, average='macro')


In [ ]:
drift_roc_auc = roc_auc_score(y_test, y_prob_drift, multi_class='ovr')


In [ ]:
print("\nDrift Noise Test Results for GaussianNB")
print("Drift strength:", drift_strength)
print("F1:", drift_f1)
print("PR-AUC:", drift_pr_auc)
print("ROC-AUC:", drift_roc_auc)
print("Inference latency:", inference_latency)


Drift Noise Test Results for GaussianNB
Drift strength: 0.2
F1: 0.012308327663151718
PR-AUC: 0.4436972285927976
ROC-AUC: 0.7004222996100484
Inference latency: 2.060556411743164e-06


In [ ]:
#Adversarial FGSM

In [ ]:
epsilon = 0.2

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

In [ ]:
model_nb_probs = model_nb.predict_proba(X_test)[:, 1]


In [ ]:
direction = np.where(model_nb_probs > 0.5, 1, -1).reshape(-1, 1)

In [ ]:
perturbation = epsilon * direction


In [ ]:
X_test_adv = X_test + perturbation


In [ ]:
#y_pred_adv = model_nb.predict(X_test_adv)


In [ ]:
start = time.time()
y_pred_adv = model_nb.predict(X_test_adv)
inference_time_total = time.time() - start
inference_latency = inference_time_total / X_test_adv.shape[0]

In [ ]:
adv_f1 = f1_score(y_test, y_pred_adv, average='weighted')


In [ ]:
y_prob_adv = model_nb.predict_proba(X_test_adv)


In [ ]:
adv_roc_auc = roc_auc_score(y_test, y_prob_adv, multi_class="ovr")


In [ ]:
adv_pr_auc = average_precision_score(y_test, y_prob_adv, average="macro")


In [ ]:
print("\nAdversarial Noise Test Results for GaussianNB")
print("Epsilon:", epsilon)
print("F1:", adv_f1)
print("PR-AUC:", adv_pr_auc)
print("ROC-AUC:", adv_roc_auc)
print("Inference latency:", inference_latency)


Adversarial Noise Test Results for GaussianNB
Epsilon: 0.2
F1: 0.012308327663151718
PR-AUC: 0.4436972285927976
ROC-AUC: 0.7004222996100484
Inference latency: 3.1890153884887696e-06


In [ ]:
#MLP noise tests

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [ ]:
dataFrame3 = pd.read_csv("/content/Danmini/combined_danmini.csv")

In [ ]:
X = dataFrame3.drop(columns="label", axis=1)
y = dataFrame3["label"]

In [ ]:
le = LabelEncoder()
y_encoded = le.fit_transform(y)
num_classes = len(np.unique(y_encoded))
y_onehot = np.eye(num_classes)[y_encoded]

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_onehot, test_size=0.2, random_state=42, stratify=y_onehot
)

In [ ]:
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(num_classes, activation='softmax')
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
#Training part

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
start_train = time.time()
history = model.fit(
    X_train, y_train,
    validation_split=0.1,
    epochs=50,
    batch_size=128,
    callbacks=[early_stop],
    verbose=2
)
train_time = time.time() - start_train

Epoch 1/50
563/563 - 7s - 12ms/step - accuracy: 0.9869 - loss: 0.0530 - val_accuracy: 0.9994 - val_loss: 0.0063
Epoch 2/50
563/563 - 3s - 5ms/step - accuracy: 0.9991 - loss: 0.0084 - val_accuracy: 0.9994 - val_loss: 0.0088
Epoch 3/50
563/563 - 3s - 6ms/step - accuracy: 0.9995 - loss: 0.0041 - val_accuracy: 0.9996 - val_loss: 0.0139
Epoch 4/50
563/563 - 2s - 3ms/step - accuracy: 0.9995 - loss: 0.0048 - val_accuracy: 0.9996 - val_loss: 0.0026
Epoch 5/50
563/563 - 2s - 3ms/step - accuracy: 0.9997 - loss: 0.0032 - val_accuracy: 0.9999 - val_loss: 0.0012
Epoch 6/50
563/563 - 2s - 3ms/step - accuracy: 0.9997 - loss: 0.0018 - val_accuracy: 0.9998 - val_loss: 0.0012
Epoch 7/50
563/563 - 2s - 3ms/step - accuracy: 0.9999 - loss: 0.0011 - val_accuracy: 0.9998 - val_loss: 7.4314e-04
Epoch 8/50
563/563 - 3s - 5ms/step - accuracy: 0.9998 - loss: 9.7291e-04 - val_accuracy: 0.9998 - val_loss: 7.8446e-04
Epoch 9/50
563/563 - 3s - 5ms/step - accuracy: 0.9997 - loss: 0.0026 - val_accuracy: 0.9999 - val_l

In [ ]:
#Gaussian Noise for MLP

In [ ]:
noise_level = 0.2
noise = np.random.normal(0, noise_level, X_test.shape)
X_test_gauss = X_test + noise

In [ ]:
start_infer = time.time()
y_pred = model.predict(X_test_gauss)
inference_time = time.time() - start_infer
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step


In [ ]:
gauss_f1 = f1_score(y_test_classes, y_pred_classes, average='weighted')

In [ ]:
gauss_roc_auc = roc_auc_score(y_test, y_pred, multi_class="ovr")

In [ ]:
gauss_pr_auc = average_precision_score(y_test, y_pred)

In [ ]:
inference_latency = (inference_time / X_test_gauss.shape[0]) * 1000


In [ ]:
print("\nGaussian Noise Test Results for MLP ")
print("Noise level:", noise_level)
print("F1:", gauss_f1)
print("PR-AUC:", gauss_pr_auc)
print("ROC-AUC:", gauss_roc_auc)
print("Inference latency:", inference_latency)



Gaussian Noise Test Results for MLP 
Noise level: 0.2
F1: 0.9997998810119377
PR-AUC: 0.999996301924866
ROC-AUC: 0.9999995154000866
Inference latency: 0.14135819673538208


In [ ]:
#Drift for MLP

In [ ]:
drift_strength = 0.2
X_test_drift = X_test + drift_strength

In [ ]:
start_infer = time.time()
y_pred = model.predict(X_test_drift)
inference_time = time.time() - start_infer
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step


In [ ]:
drift_f1 = f1_score(y_test_classes, y_pred_classes, average='weighted')

In [ ]:
drift_roc_auc = roc_auc_score(y_test, y_pred, multi_class="ovr")

In [ ]:
drift_pr_auc = average_precision_score(y_test, y_pred)

In [ ]:
inference_latency = (inference_time / X_test_drift.shape[0]) * 1000


In [ ]:
print("\nDrift Noise Test Results for MLP ")
print("Drift :", drift_strength)
print("F1:", drift_f1)
print("PR-AUC:", drift_pr_auc)
print("ROC-AUC:", drift_roc_auc)
print("Inference latency:", inference_latency)


Drift Noise Test Results for MLP 
Drift : 0.2
F1: 0.9635685324528193
PR-AUC: 0.9999969941099439
ROC-AUC: 0.9999814877414664
Inference latency: 0.2983981132507324


In [ ]:
#Adversarial FGSM on MLP

In [ ]:
epsilon = 0.2

In [ ]:
probs = model.predict(X_test)

625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [ ]:
pred_classes = np.argmax(probs, axis=1)


In [ ]:
direction = pred_classes.reshape(-1, 1).astype(float)
direction = np.where(direction == 0, -1, direction)
direction = np.where(direction == 1, 1, direction)
direction = np.where(direction == 2, 1, direction)

In [ ]:
direction = np.sign(direction)


In [ ]:
perturbation = epsilon * direction


In [ ]:
X_test_adv = X_test + perturbation


In [ ]:
y_pred_adv_probs = model.predict(X_test_adv)


625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step


In [ ]:
y_pred_adv = np.argmax(y_pred_adv_probs, axis=1)


In [ ]:
y_test_classes = np.argmax(y_test, axis=1)


In [ ]:
adv_f1 = f1_score(y_test_classes, y_pred_adv, average='weighted')


In [ ]:
adv_roc_auc = roc_auc_score(y_test, y_pred_adv_probs, multi_class='ovr')


In [ ]:
adv_pr_auc = average_precision_score(y_test, y_pred_adv_probs)

In [ ]:
start_infer = time.time()
inference_time = time.time() - start_infer

In [ ]:
inference_latency = (inference_time / X_test_adv.shape[0]) * 1000


In [ ]:
print("\nAdversarial Noise Test Results for MLP")
print("Epsilon:", epsilon)
print("F1:", adv_f1)
print("PR-AUC:", adv_pr_auc)
print("ROC-AUC:", adv_roc_auc)
print("Inference latency:", inference_latency)


Adversarial Noise Test Results for MLP
Epsilon: 0.2
F1: 0.9634184951056157
PR-AUC: 0.9993113080173274
ROC-AUC: 0.9993113080173274
Inference latency: 4.6730041503906245e-06


In [ ]:
#The noise tests for Provision PT737E

In [ ]:
zip_path2 = "/content/combined_ProvisionPT737E.zip"

In [ ]:
extract_path2 = "/content/Provision"

In [ ]:
if not Path(extract_path2).exists():
    with zipfile.ZipFile(zip_path2, 'r') as zip_ref:
        zip_ref.extractall(extract_path2)
    print("Dataset unzipped.")
else:
    print("Dataset already unzipped.")

Dataset unzipped.


In [ ]:
dataFrame4 = pd.read_csv("/content/Provision/combined_ProvisionPT737E.csv")


In [ ]:
#Random forest for Provision Camera

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
X = dataFrame4.drop(columns="label", axis=1)
y = dataFrame4["label"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

In [ ]:
rf2 = RandomForestClassifier(
    n_estimators=200,
    max_depth=None,
    random_state=42,
    n_jobs=-1
)

In [ ]:
start_train = time.time()
rf2.fit(X_train, y_train)
end_train = time.time()


print(f"\nTraining time: {end_train - start_train:.3f} seconds")


Training time: 55.059 seconds


In [ ]:
#Gaussian Noise test

In [ ]:
noise = 0.2

In [ ]:
X_test_gaussian = X_test + np.random.normal(0, noise, X_test.shape)


In [ ]:
y_pred_gauss = rf2.predict(X_test_gaussian)
y_prob_gauss = rf2.predict_proba(X_test_gaussian)

In [ ]:
gauss_f1 = f1_score(y_test, y_pred_gauss, average="weighted")

In [ ]:
gauss_pr_auc = average_precision_score(y_test, y_prob_gauss, average='macro')

In [ ]:
gauss_roc_auc = roc_auc_score(y_test, y_prob_gauss, multi_class='ovr')


In [ ]:
start = time.time()
y_pred = rf2.predict(X_test_gaussian)
inference_time_total = time.time() - start
inference_latency_per_sample = inference_time_total / X_test_gaussian.shape[0]

In [ ]:
print("Gaussian Noise Test Results for Random Forest ")
print("Noise " , noise)
print("F1 Score:", gauss_f1)
print("PR AUC:", gauss_pr_auc)
print("ROC AUC:", gauss_roc_auc)
print("Interference Latency: ", inference_latency_per_sample)

Gaussian Noise Test Results for Random Forest 
Noise  0.2
F1 Score: 0.17264279941453667
PR AUC: 0.9999988471630977
ROC AUC: 0.9999997100366343
Interference Latency:  1.8257435162862143e-05


In [ ]:
#Drift noise test

In [ ]:
drift_strength = 0.2

In [ ]:
X_test_drift = X_test + drift_strength


In [ ]:
y_pred_drift = rf2.predict(X_test_drift)
y_prob_drift = rf2.predict_proba(X_test_drift)

In [ ]:
drift_f1 = f1_score(y_test, y_pred_drift, average="weighted")


In [ ]:
drift_pr_auc = average_precision_score(y_test, y_prob_drift, average='macro')

In [ ]:
drift_roc_auc = roc_auc_score(y_test, y_prob_drift, multi_class='ovr')

In [ ]:
start = time.time()
y_pred = rf2.predict(X_test_drift)
inference_time_total = time.time() - start
inference_latency_per_sample = inference_time_total / X_test_drift.shape[0]

In [ ]:
print("Drift Noise Test Results for Random Forest ")
print("Drift strength " , drift_strength)
print("F1 Score:", drift_f1)
print("PR AUC:", drift_pr_auc)
print("ROC AUC:", drift_roc_auc)
print("Interference Latency: ", inference_latency_per_sample)

Drift Noise Test Results for Random Forest 
Drift strength  0.2
F1 Score: 0.13604843164979968
PR AUC: 1.0
ROC AUC: 1.0
Interference Latency:  1.3266658782958984e-05


In [ ]:
#Adversarial FGSM noise test

In [ ]:
epsilon = 0.2

In [ ]:
rf_probs = rf2.predict_proba(X_test)[:, 1]


In [ ]:
direction = np.where(rf_probs > 0.5, 1, -1).reshape(-1, 1)

In [ ]:
perturbation = epsilon * direction


In [ ]:
X_test_adv = X_test + perturbation


In [ ]:
y_pred_adv = rf2.predict(X_test_adv)

In [ ]:
adv_f1 = f1_score(y_test, y_pred_adv, average='weighted')


In [ ]:
y_prob_adv = rf2.predict_proba(X_test_adv)


In [ ]:
adv_roc_auc = roc_auc_score(y_test, y_prob_adv, multi_class="ovr")


In [ ]:
adv_pr_auc = average_precision_score(y_test, y_prob_adv, average="macro")


In [ ]:
start = time.time()
y_pred = rf2.predict(X_test_adv)
inference_time_total = time.time() - start
inference_latency_per_sample = inference_time_total / X_test_adv.shape[0]

In [ ]:
print("Adversarial Noise Test Results for Random Forest ")
print("Epsilon " , epsilon)
print("F1 Score:", adv_f1)
print("PR AUC:", adv_pr_auc)
print("ROC AUC:", adv_roc_auc)
print("Interference Latency: ", inference_latency_per_sample)

Adversarial Noise Test Results for Random Forest 
Epsilon  0.2
F1 Score: 0.3649396894310931
PR AUC: 1.0
ROC AUC: 1.0
Interference Latency:  1.674030621846517e-05


In [ ]:
#Gaussian NB for Provision camera

In [ ]:
dataFrame5 = pd.read_csv("/content/Provision/combined_ProvisionPT737E.csv")


In [ ]:
X = dataFrame5.drop(columns="label", axis=1)
y = dataFrame5["label"]

In [ ]:
from sklearn.naive_bayes import GaussianNB


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

In [ ]:
model_nb2 = GaussianNB()

In [ ]:
model_nb2.fit(X_train, y_train)

GaussianNB()

In [ ]:
#Gaussian noise for gaussian naive bayes

In [ ]:
noise_level = 0.2
noise = np.random.normal(0, noise_level, X_test.shape)
X_test_gauss = X_test + noise

In [ ]:
start = time.time()
y_pred_gauss = model_nb2.predict(X_test_gauss)
inference_time_total = time.time() - start
inference_latency = inference_time_total / X_test_gauss.shape[0]

In [ ]:
y_prob_gauss = model_nb2.predict_proba(X_test_gauss)

In [ ]:
gauss_f1 = f1_score(y_test, y_pred_gauss, average="weighted")

In [ ]:
gauss_pr_auc = average_precision_score(y_test, y_prob_gauss, average='macro')


In [ ]:
gauss_roc_auc = roc_auc_score(y_test, y_prob_gauss, multi_class='ovr')

In [ ]:
print("\nGaussian Noise Test Results for GaussianNB")
print("Noise level:", noise_level)
print("F1:", gauss_f1)
print("PR-AUC:", gauss_pr_auc)
print("ROC-AUC:", gauss_roc_auc)
print("Inference latency:", inference_latency)


Gaussian Noise Test Results for GaussianNB
Noise level: 0.2
F1: 0.025916212001211483
PR-AUC: 0.4414590821851987
ROC-AUC: 0.6700915300463529
Inference latency: 4.0878057479858395e-06


In [ ]:
#Drift noise checks

In [ ]:
drift_strength = 0.2
X_test_drift = X_test + drift_strength

In [ ]:
y_prob_drift = model_nb2.predict_proba(X_test_drift)

In [ ]:
start = time.time()
y_pred_drift = model_nb2.predict(X_test_drift)
inference_time_total = time.time() - start
inference_latency = inference_time_total / X_test_drift.shape[0]

In [ ]:
drift_f1 = f1_score(y_test, y_pred_drift, average="weighted")


In [ ]:
drift_pr_auc = average_precision_score(y_test, y_prob_drift, average='macro')

In [ ]:
drift_roc_auc = roc_auc_score(y_test, y_prob_drift, multi_class='ovr')


In [ ]:
print("\nDrift Noise Test Results for GaussianNB")
print("Drift strength:", drift_strength)
print("F1:", drift_f1)
print("PR-AUC:", drift_pr_auc)
print("ROC-AUC:", drift_roc_auc)
print("Inference latency:", inference_latency)


Drift Noise Test Results for GaussianNB
Drift strength: 0.2
F1: 0.025916212001211483
PR-AUC: 0.4412059963849779
ROC-AUC: 0.6700915300463529
Inference latency: 2.7435302734375e-06


In [ ]:
#Adversarial FGSM for Gaussian Naive Bayes

In [ ]:
epsilon = 0.2

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

In [ ]:
model_nb_probs = model_nb2.predict_proba(X_test)[:, 1]


In [ ]:
direction = np.where(model_nb_probs > 0.5, 1, -1).reshape(-1, 1)

In [ ]:
perturbation = epsilon * direction


In [ ]:
X_test_adv = X_test + perturbation


In [ ]:
start = time.time()
y_pred_adv = model_nb2.predict(X_test_adv)
inference_time_total = time.time() - start
inference_latency = inference_time_total / X_test_adv.shape[0]

In [ ]:
adv_f1 = f1_score(y_test, y_pred_adv, average='weighted')

In [ ]:
y_prob_adv = model_nb2.predict_proba(X_test_adv)


In [ ]:
adv_roc_auc = roc_auc_score(y_test, y_prob_adv, multi_class="ovr")

In [ ]:
adv_pr_auc = average_precision_score(y_test, y_prob_adv, average="macro")


In [ ]:
print("\nAdversarial Noise Test Results for GaussianNB")
print("Epsilon:", epsilon)
print("F1:", adv_f1)
print("PR-AUC:", adv_pr_auc)
print("ROC-AUC:", adv_roc_auc)
print("Inference latency:", inference_latency)


Adversarial Noise Test Results for GaussianNB
Epsilon: 0.2
F1: 0.025916212001211483
PR-AUC: 0.4412059963849779
ROC-AUC: 0.6700915300463529
Inference latency: 7.778612772623698e-06


In [ ]:
dataFrame6 = pd.read_csv("/content/Provision/combined_ProvisionPT737E.csv")

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [ ]:
X = dataFrame6.drop(columns="label", axis=1)
y = dataFrame6["label"]

In [ ]:
le = LabelEncoder()
y_encoded = le.fit_transform(y)
num_classes = len(np.unique(y_encoded))
y_onehot = np.eye(num_classes)[y_encoded]

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_onehot, test_size=0.2, random_state=42, stratify=y_onehot
)

In [ ]:
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(num_classes, activation='softmax')
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
#Training

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
start_train = time.time()
history = model.fit(
    X_train, y_train,
    validation_split=0.1,
    epochs=50,
    batch_size=128,
    callbacks=[early_stop],
    verbose=2
)
train_time = time.time() - start_train

Epoch 1/50
563/563 - 7s - 13ms/step - accuracy: 0.9885 - loss: 0.0379 - val_accuracy: 0.9995 - val_loss: 0.0067
Epoch 2/50
563/563 - 2s - 3ms/step - accuracy: 0.9992 - loss: 0.0043 - val_accuracy: 0.9995 - val_loss: 0.0069
Epoch 3/50
563/563 - 2s - 3ms/step - accuracy: 0.9992 - loss: 0.0045 - val_accuracy: 0.9998 - val_loss: 0.0070
Epoch 4/50
563/563 - 2s - 3ms/step - accuracy: 0.9997 - loss: 0.0024 - val_accuracy: 0.9996 - val_loss: 0.0087
Epoch 5/50
563/563 - 3s - 5ms/step - accuracy: 0.9995 - loss: 0.0043 - val_accuracy: 0.9998 - val_loss: 0.0047
Epoch 6/50
563/563 - 4s - 8ms/step - accuracy: 0.9998 - loss: 0.0015 - val_accuracy: 0.9998 - val_loss: 0.0046
Epoch 7/50
563/563 - 2s - 3ms/step - accuracy: 0.9997 - loss: 0.0017 - val_accuracy: 0.9998 - val_loss: 0.0032
Epoch 8/50
563/563 - 2s - 3ms/step - accuracy: 0.9997 - loss: 0.0011 - val_accuracy: 0.9999 - val_loss: 0.0025
Epoch 9/50
563/563 - 2s - 3ms/step - accuracy: 0.9998 - loss: 0.0015 - val_accuracy: 0.9998 - val_loss: 0.0032


In [ ]:
#Gaussian Noise tests for MLP

In [ ]:
noise_level = 0.2
noise = np.random.normal(0, noise_level, X_test.shape)
X_test_gauss = X_test + noise

In [ ]:
start_infer = time.time()
y_pred = model.predict(X_test_gauss)
inference_time = time.time() - start_infer
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [ ]:
gauss_f1 = f1_score(y_test_classes, y_pred_classes, average='weighted')

In [ ]:
gauss_roc_auc = roc_auc_score(y_test, y_pred, multi_class="ovr")

In [ ]:
gauss_pr_auc = average_precision_score(y_test, y_pred)

In [ ]:
inference_latency = (inference_time / X_test_gauss.shape[0]) * 1000

In [ ]:
print("\nGaussian Noise Test Results for MLP ")
print("Noise level:", noise_level)
print("F1:", gauss_f1)
print("PR-AUC:", gauss_pr_auc)
print("ROC-AUC:", gauss_roc_auc)
print("Inference latency:", inference_latency)


Gaussian Noise Test Results for MLP 
Noise level: 0.2
F1: 0.9997000504860365
PR-AUC: 0.9999981661346354
ROC-AUC: 0.9999996425943435
Inference latency: 0.10742833614349366


In [ ]:
#Drift noise tests for MLP

In [ ]:
drift_strength = 0.2
X_test_drift = X_test + drift_strength

In [ ]:
start_infer = time.time()
y_pred = model.predict(X_test_drift)
inference_time = time.time() - start_infer
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [ ]:
drift_f1 = f1_score(y_test_classes, y_pred_classes, average='weighted')

In [ ]:
drift_roc_auc = roc_auc_score(y_test, y_pred, multi_class="ovr")

In [ ]:
drift_pr_auc = average_precision_score(y_test, y_pred)

In [ ]:
inference_latency = (inference_time / X_test_drift.shape[0]) * 1000


In [ ]:
print("\nDrift Noise Test Results for MLP ")
print("Drift :", drift_strength)
print("F1:", drift_f1)
print("PR-AUC:", drift_pr_auc)
print("ROC-AUC:", drift_roc_auc)
print("Inference latency:", inference_latency)


Drift Noise Test Results for MLP 
Drift : 0.2
F1: 0.9949441774151186
PR-AUC: 0.9999953782937782
ROC-AUC: 0.9999984998336539
Inference latency: 0.13517552614212036


In [ ]:
#Adversarial noise tests for MLP

In [ ]:
epsilon = 0.2

In [ ]:
probs = model.predict(X_test)

625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [ ]:
pred_classes = np.argmax(probs, axis=1)

In [ ]:
direction = pred_classes.reshape(-1, 1).astype(float)
direction = np.where(direction == 0, -1, direction)
direction = np.where(direction == 1, 1, direction)
direction = np.where(direction == 2, 1, direction)

In [ ]:
direction = np.sign(direction)

In [ ]:
perturbation = epsilon * direction

In [ ]:
X_test_adv = X_test + perturbation

In [ ]:
y_pred_adv_probs = model.predict(X_test_adv)

625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step


In [ ]:
y_pred_adv = np.argmax(y_pred_adv_probs, axis=1)

In [ ]:
y_test_classes = np.argmax(y_test, axis=1)

In [ ]:
adv_f1 = f1_score(y_test_classes, y_pred_adv, average='weighted')

In [ ]:
adv_roc_auc = roc_auc_score(y_test, y_pred_adv_probs, multi_class='ovr')


In [ ]:
adv_pr_auc = average_precision_score(y_test, y_pred_adv_probs)

In [ ]:
start_infer = time.time()
inference_time = time.time() - start_infer

In [ ]:
inference_latency = (inference_time / X_test_adv.shape[0]) * 1000

In [ ]:
print("\nAdversarial Noise Test Results for MLP")
print("Epsilon:", epsilon)
print("F1:", adv_f1)
print("PR-AUC:", adv_pr_auc)
print("ROC-AUC:", adv_roc_auc)
print("Inference latency:", inference_latency)


Adversarial Noise Test Results for MLP
Epsilon: 0.2
F1: 0.9950433489778138
PR-AUC: 0.999995234382864
ROC-AUC: 0.9999963319696135
Inference latency: 4.041194915771484e-06
